# MaLkuTeam представляет: EmoQuant

Данный модуль предназначен вывода распределения вероятностей эмоционального окраса по сегменту аудиодорожки, который выделятся из всего аудио, полученного благодаря tg-боту видео. Аудиодорожка делится на сегменты по спикерам на основании времени начала (1) и конца (2) фразы конкретного говорящего. (1) и (2) берутся из словоря модуля для транскрибирования и диаризации. Соответственно, благодаря связки этих двух модулей можно получить для каждого спикера последовательность распределений эмоций по всей встрече. Данную информацию можно в дальнейшем анализировать при помощи стандартных инструментов (pandas, matplotlib), либо при помощи других моделей машинного обучения.

*Далее представлен сам код и его описание*

1. Устанавливаем wget для загрузки весов модели и другие библиотеки + устанавливаем связь с библиотекой NeMo


In [11]:
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install omegaconf
BRANCH = 'r1.21.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2ubuntu0.1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
sox is already the newest version (14.4.2+git20190427-2+deb11u2ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
DEPRECATION: git+https://github.com/NVIDIA/NeMo.git@r1.21.0#egg=nemo_toolkit[all] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Cloning https://github.com/NVIDIA/NeMo.git (to revision r1.21.0) to /tmp/pip-install-00mgbl71/nemo-toolkit_6ac5406b12e54196b43146289c6a1c2a
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git /tmp/pip-install-00mgbl71/nemo-toolkit_6ac5406b12e54196b43146289c6a1c2

2. Подгружаем оставшиеся библиотеки

In [12]:
from typing import List, Union

import torch
import torchaudio
import soundfile as sf
from omegaconf import DictConfig, ListConfig, OmegaConf
import hydra

3. Создаем класс модели

In [13]:
class SpecScaler(torch.nn.Module):
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return torch.log(x.clamp_(1e-9, 1e9))


class GigaAMEmo(torch.nn.Module):
    def __init__(self, conf: Union[DictConfig, ListConfig]):
        super().__init__()
        self.id2name = conf.id2name
        self.feature_extractor = hydra.utils.instantiate(conf.feature_extractor)
        self.conformer = hydra.utils.instantiate(conf.encoder)
        self.linear_head = hydra.utils.instantiate(conf.classification_head)

    def forward(self, features, features_length=None):
        if features.dim() == 2:
            features = features.unsqueeze(0)
        if not features_length:
            features_length = torch.ones(features.shape[0]) * features.shape[-1]
            features_length = features_length.to(features.device)
        encoded, _ = self.conformer(audio_signal=features, length=features_length)
        encoded_pooled = torch.nn.functional.avg_pool1d(
            encoded, kernel_size=encoded.shape[-1]
        ).squeeze(-1)

        logits = self.linear_head(encoded_pooled)
        return logits

    def get_probs(self, audio_path: str) -> List[List[float]]:
        audio_signal, _ = sf.read(audio_path, dtype="float32")
        print(torch.tensor(audio_signal).shape)
        features = self.feature_extractor(torch.tensor(audio_signal)[:1500000,0].float().to(next(self.parameters()).device))
        logits = self.forward(features)
        probs = torch.nn.functional.softmax(logits).detach().tolist()
        return probs

4. Скачиваем веса и конфиг

In [9]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!wget https://n-ws-q0bez.s3pd12.sbercloud.ru/b-ws-q0bez-jpv/GigaAM/emo_model_weights.ckpt
!wget https://n-ws-q0bez.s3pd12.sbercloud.ru/b-ws-q0bez-jpv/GigaAM/emo_model_config.yaml

--2024-04-21 07:35:47--  https://n-ws-q0bez.s3pd12.sbercloud.ru/b-ws-q0bez-jpv/GigaAM/emo_model_weights.ckpt
Resolving n-ws-q0bez.s3pd12.sbercloud.ru (n-ws-q0bez.s3pd12.sbercloud.ru)... 37.230.193.192
Connecting to n-ws-q0bez.s3pd12.sbercloud.ru (n-ws-q0bez.s3pd12.sbercloud.ru)|37.230.193.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 968409626 (924M) [application/octet-stream]
Saving to: ‘emo_model_weights.ckpt’

emo_model_weights.c 100%[===================>] 923.55M  7.35MB/s    in 1m 49s  

2024-04-21 07:37:39 (8.48 MB/s) - ‘emo_model_weights.ckpt’ saved [968409626/968409626]

--2024-04-21 07:37:39--  https://n-ws-q0bez.s3pd12.sbercloud.ru/b-ws-q0bez-jpv/GigaAM/emo_model_config.yaml
Resolving n-ws-q0bez.s3pd12.sbercloud.ru (n-ws-q0bez.s3pd12.sbercloud.ru)... 37.230.193.192
Connecting to n-ws-q0bez.s3pd12.sbercloud.ru (n-ws-q0bez.s3pd12.sbercloud.ru)|37.230.193.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 765 [applicatio

5. Синхронизируем все выше перечисленное для обработки примера. Вывод - распределение вероятностей по эмоциям

In [14]:
model_config = 'emo_model_config.yaml'
model_weights = 'emo_model_weights.ckpt'
audio_path = '/content/APENT_-_Mozhno_ya_s_tobojj_77552797.mp3'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

conf = OmegaConf.load(model_config)
model = GigaAMEmo(conf)
ckpt = torch.load(model_weights, map_location="cpu")
model.load_state_dict(ckpt, strict=False)
model = model.to(device)
model.eval()
with torch.no_grad():
    probs = model.get_probs(audio_path)[0]
print('angry: ',probs[0],'sad: ',probs[1],'neutal: ',probs[2],'positive: ',probs[3])

torch.Size([5836032, 2])


[NeMo W 2024-04-21 09:44:27 nemo_logging:349] <ipython-input-13-5347fadbc477>:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
      probs = torch.nn.functional.softmax(logits).detach().tolist()
    


angry:  0.012140360660851002 sad:  0.6520755290985107 neutal:  0.25892260670661926 positive:  0.07686156034469604


В переменную audio_path записывается имя файла в формате mp3, который в идеале извлекается из видео. В коде записан файл, на котором мы тестировали модель.